# 第7章 方差分析

## 7.1方差分析的基本原理

>比较两个样本的方差是否相等，用F统计量

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

path=r"D:\文档\Python Scripts\Python数据分析基础（第2版）数据\ch7\dc_sales.csv"
data=pd.read_csv(path)
print(data.head())
data['pixel']=data['pixel'].astype('category')
data.cat.categories=['<500','500-600','600-800','800-1000','>1000']
print(pd.pivot_table(data,index='pixel',columns='market',values='sales'))


   market  pixel  sales
0       1      1     70
1       1      2    101
2       1      3    114
3       1      4    120
4       1      5    132
